<a href="https://colab.research.google.com/github/F3binThomas/AI-Agent-/blob/main/ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Colab Cell 1: Installation and Setup ---

# 1. Install necessary libraries
!pip install google-genai pydantic

# 2. Securely load the API Key from Colab Secrets (GEMINI_API_KEY)
# This assumes you have already set the secret in the Colab Secrets panel (the key icon).
import os
from google.colab import userdata
from google import genai

try:
    os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    # Initialize the Gemini Client
    client = genai.Client()
    print("✅ Setup Complete: Dependencies installed and Gemini Client initialized.")
except Exception as e:
    print(f"🔴 FATAL ERROR during setup: {e}")
    print("Please ensure the 'GEMINI_API_KEY' secret is correctly set in the Colab Secrets panel.")

✅ Setup Complete: Dependencies installed and Gemini Client initialized.


In [ ]:
# --- Colab Cell 2: Logic, Schema, and Data Stubs ---

from pydantic import BaseModel, Field
import json
from google.genai import types

# --- 1. CUSTOM TOOL FUNCTION (Auditable Business Logic - Day 2) ---
def calculate_match_score(resume_text: str, job_description_text: str) -> float:
    """
    TOOL: Calculates a quantitative match score (0-100) between a resume and job description.
    This simulates integration with a deterministic, external scoring system.
    """
    if not job_description_text:
        return 0.0

    # Standardize and prepare texts
    resume_words = set(resume_text.lower().split())
    job_words = set(job_description_text.lower().split())

    # Define Key Skills/Requirements to prioritize (30% Weight)
    key_requirements = ['python', 'machine learning', 'sql', 'agentic', 'gemini', 'workflow', 'data analysis', 'pydantic']

    # --- Scoring Calculation ---

    # 1. Base Overlap Score (General Relevance - 70% Weight)
    common_words = resume_words.intersection(job_words)
    base_overlap_ratio = len(common_words) / len(job_words) if len(job_words) > 0 else 0

    # 2. Key Skill Bonus (High-Value Match - 30% Weight)
    key_match_count = sum(1 for req in key_requirements if req in resume_words)
    key_match_ratio = key_match_count / len(key_requirements)

    # Combine and scale to 100
    final_score = (base_overlap_ratio * 0.70) + (key_match_ratio * 0.30)

    return round(min(final_score * 100, 100.0), 2)


# --- 2. STRUCTURED OUTPUT SCHEMA (Evaluation and Auditing - Day 4) ---
class AnalysisResult(BaseModel):
    """
    Structured analysis and feedback generated by the Analyzer Agent.
    Used to enforce a predictable and parsable output structure.
    """
    match_score: float = Field(
        description="The quantitative match score (0-100) returned by the 'calculate_match_score' tool."
    )
    strongest_skills_matched: list[str] = Field(
        description="A list of 3-5 specific technical or soft skills from the resume that best align with the job requirements."
    )
    suggestions_for_improvement: list

In [ ]:
# --- Colab Cell 3: FINAL ROBUST Orchestrator Agent Function Definition ---

# The list of tools available to the Analyzer Agent
TOOLS_LIST = [calculate_match_score]

# The main function that orchestrates the entire agent workflow
def run_application_agent(resume_text: str, job_description_text: str, client: genai.Client):
    """
    Orchestrates the robust two-step Enterprise Agent workflow, relying on prompt engineering
    for structured output to avoid API constraints.
    """
    if not client:
        print("🔴 ERROR: Gemini client not initialized. Cannot run agent.")
        return

    # --- STEP 1: ANALYZER AGENT (Tool Use and Prompt-Based Structured Output) ---
    print("\n\n--- 🤖 STEP 1: ANALYZER AGENT (Scoring & Feedback) ---")

    # 1.1. Construct the Analyzer Prompt - CRITICAL FIX HERE: Force JSON text output
    analyzer_prompt = f"""
    You are the **Candidate Analyzer Agent**. Your sole task is to rigorously evaluate the provided resume
    against the job description.

    FIRST: You MUST use the available function 'calculate_match_score' to get the quantitative score.
    SECOND: Based on the score and your analysis of the texts, you MUST generate a response that is
    **ONLY and EXACTLY a JSON object** that conforms to the following schema:

    {{
        "match_score": <The score from the tool>,
        "strongest_skills_matched": ["skill1", "skill2", "skill3"],
        "suggestions_for_improvement": ["suggestion1", "suggestion2", "suggestion3"]
    }}

    Do not output any introductory text, commentary, or text outside of the raw JSON object.

    --- INPUT DATA ---
    <RESUME>
    {resume_text}
    </RESUME>

    <JOB_DESCRIPTION>
    {job_description_text}
    </JOB_DESCRIPTION>
    """

    # 1.2. Call the Model with Tool Use (gemini-2.5-pro for better adherence)
    try:
        analyzer_response = client.models.generate_content(
            model='gemini-2.5-pro', # Use Pro model for stability and adherence
            contents=analyzer_prompt,
            config=types.GenerateContentConfig(
                tools=TOOLS_LIST,  # Makes the tool available (Day 2)
                # CRITICAL FIX: Removing the problematic response_mime_type and response_schema
            ),
        )

        # 1.3. Parse the Raw Text Output (rely on model adherence)
        # This will raise a JSONDecodeError if the model doesn't follow the JSON instruction
        analysis_data = json.loads(analyzer_response.text)
        analysis_result = AnalysisResult(**analysis_data)

        print(f"✅ Analyzer Agent Success. Structured output parsed.")

        # --- Store result in memory/context for next agent (Day 3: State) ---
        analysis_context = analysis_result.model_dump_json(indent=2)


    except json.JSONDecodeError:
        print(f"🔴 ERROR in Analyzer Agent: Model did not return valid JSON. Response text: {analyzer_response.text}")
        return
    except Exception as e:
        print(f"🔴 ERROR in Analyzer Agent: {e}")
        return

    # --- STEP 2: GENERATOR AGENT (Context-Aware Creation) ---
    print("\n\n--- ✍️ STEP 2: COVER LETTER GENERATOR AGENT ---")

    # The rest of the Generator Agent logic remains the same (Steps 2.1 to 2.3)
    generator_prompt = f"""
    You are the **Cover Letter Generator Agent**. Your task is to draft a professional,
    highly personalized, and compelling cover letter for the candidate applying for the job.

    The tone should be enthusiastic and professional. The letter MUST specifically call out
    at least three skills listed in the 'strongest_skills_matched' section below to show deep relevance.

    --- INPUT CONTEXT (Memory from Analyzer Agent) ---
    {analysis_context}

    --- INPUT DATA ---
    <RESUME>
    {resume_text}
    </RESUME>

    <JOB_DESCRIPTION>
    {job_description_text}
    </JOB_DESCRIPTION>
    """

    # 2.2. Call the Model for Generation
    try:
        generator_response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=generator_prompt,
            config=types.GenerateContentConfig(
                temperature=0.7
            )
        )

        print("✅ Generator Agent Success. Outputting Final Report...")

        # 2.3. FINAL ENTERPRISE REPORT
        print("\n" + "="*50)
        print("       **FINAL ENTERPRISE APPLICATION REPORT**")
        print("="*50)

        print("\n### 📈 Candidate Match Summary")
        print(f"**MATCH SCORE (Auditable Metric):** {analysis_result.match_score}%")
        print(f"**Strongest Match Areas:** {', '.join(analysis_result.strongest_skills_matched)}")

        print("\n### 🎯 Suggestions for Improvement (Pre-Interview Prep)")
        for i, suggestion in enumerate(analysis_result.suggestions_for_improvement):
            print(f"{i+1}. {suggestion}")

        print("\n### 💌 Personalized Cover Letter")
        print("-" * 30)
            # Safely replace any surrounding ticks the model might add
        final_letter = generator_response.text.strip().replace("```json\n", "").replace("```", "")
        print(final_letter)
        print("-" * 30)

        print("\n" + "="*50)
        print("       **END OF REPORT**")
        print("="*50)


    except Exception as e:
        print(f"🔴 ERROR in Generator Agent: {e}")
        return

In [ ]:
# --- Colab Cell 4: Final Execution ---

if __name__ == '__main__':
    print("--- 🚀 RUNNING ENTERPRISE JOB APPLICATION AGENT ---")

    # Pass the sample data and the initialized client to start the process
    run_application_agent(SAMPLE_RESUME, SAMPLE_JOB_DESCRIPTION, client)

--- 🚀 RUNNING ENTERPRISE JOB APPLICATION AGENT ---


--- 🤖 STEP 1: ANALYZER AGENT (Scoring & Feedback) ---
✅ Analyzer Agent Success. Structured output parsed.


--- ✍️ STEP 2: COVER LETTER GENERATOR AGENT ---
✅ Generator Agent Success. Outputting Final Report...

       **FINAL ENTERPRISE APPLICATION REPORT**

### 📈 Candidate Match Summary
**MATCH SCORE (Auditable Metric):** 92.0%
**Strongest Match Areas:** 5+ years of experience in Python and AI/ML, Agentic workflows and Large Language Models (LLMs), Experience with Gemini API and Pydantic, Data analysis and SQL for data pipelines, Experience with AWS cloud platform

### 🎯 Suggestions for Improvement (Pre-Interview Prep)
1. Explicitly mention experience with 'distributed systems', possibly by expanding on the Apache Kafka project.
2. Add details about leadership or mentorship roles to align with the 'Mentor junior developers' responsibility.
3. Include any experience with 'rigorous testing and validation' of AI/ML models, as this is a